# How cells are connected ?

### Importation of all libraries and modules needed

In [118]:
from skimage.filters import threshold_yen
from skimage import filters
from skimage import morphology
import skimage
from skimage import data
from skimage.segmentation import clear_border
import napari
import matplotlib.pyplot as plt
from tifffile import imread
import tifffile
from skimage import restoration
import skimage.filters as filters
from skimage.morphology import disk
from scipy import ndimage as ndi
from skimage.segmentation import clear_border
import numpy as np
from skimage import morphology
import napari
from magicgui import magicgui
from napari.types import ImageData, LabelsData
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.segmentation import watershed
from skimage import measure
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import glob
from skimage.morphology import skeletonize

In [213]:
#after cleaning the csv file by hand (by deleted the lines that contains files that was not in the list of images
    #of FDV_cell_database, and by deleting the images that we choose to delete due to bad luminosity, and too hard
    #to process), we open the cleaned csv file as a dataframe containing all the names, categories, etc, of the 
    #cell images.
def csv_to_df(csv,df):
    #csv need to be a file name bewteen ""
    
    df = pd.read_csv(csv)
    return df

In [214]:
csv_to_df('FDV_database/global_database_cleaned.csv',global_df_cleaned)

,plant_name,cell_fname,leaf_id,leaf_orientation,categories
0,Ceratostigma_minus,3-07-400x-1-01,3.70,abaxial,stomates
1,Ceratostigma_minus,3-07-400x-1-02,3.70,abaxial,stomates
2,Ceratostigma_minus,3-07-400x-1-03,3.70,abaxial,stomates
3,Ceratostigma_minus,3-07-400x-1-04,3.70,abaxial,stomates
4,Ceratostigma_minus,3-07-400x-2-01,3.70,adaxial,stomates
...,...,...,...,...,...
118,Alstroemeria_aurea,7-23-400x-1-01,7.23,abaxial,NaN
119,Alstroemeria_aurea,7-23-400x-1-03,7.23,abaxial,NaN
120,Alstroemeria_aurea,7-23-400x-1-04,7.23,abaxial,NaN
121,Alstroemeria_aurea,7-23-400x-1-05,7.23,abaxial,NaN


In [215]:
#We sort the dataframe by ascending values in the column "categories" (which contains the names of the categories
    #we chose to sort the images and clean our data.)

def sort_df(df):
    
    df = df.sort_values("categories")
    return df

In [216]:
sort_df(global_df_cleaned)

,plant_name,cell_fname,leaf_id,leaf_orientation,categories
72,Alstroemeria_aurea,7-23-400x-2-05,7.23,adaxial,croco
71,Alstroemeria_aurea,7-23-400x-2-04,7.23,adaxial,croco
70,Alstroemeria_aurea,7-23-400x-2-03,7.23,adaxial,croco
69,Alstroemeria_aurea,7-23-400x-2-02,7.23,adaxial,croco
68,Alstroemeria_aurea,7-23-400x-2-01,7.23,adaxial,croco
...,...,...,...,...,...
118,Alstroemeria_aurea,7-23-400x-1-01,7.23,abaxial,NaN
119,Alstroemeria_aurea,7-23-400x-1-03,7.23,abaxial,NaN
120,Alstroemeria_aurea,7-23-400x-1-04,7.23,abaxial,NaN
121,Alstroemeria_aurea,7-23-400x-1-05,7.23,abaxial,NaN


In [245]:
#The deleted cell images have NaN values in the column "categories". We delet them here by deleting the lines 
    #containing NaN in the column "categories" using the panda function .dropna.
    
def clean_df(df1,df2,column_name):
    #column_name need to be a name of a column of the dataframe df between ""
    
    df2 = df1.dropna(subset=[column_name]) 
    return df2

In [246]:
clean_df(global_df_cleaned,global_df_cleaned2,'categories')

,plant_name,cell_fname,leaf_id,leaf_orientation,categories
0,Ceratostigma_minus,3-07-400x-1-01,3.70,abaxial,stomates
1,Ceratostigma_minus,3-07-400x-1-02,3.70,abaxial,stomates
2,Ceratostigma_minus,3-07-400x-1-03,3.70,abaxial,stomates
3,Ceratostigma_minus,3-07-400x-1-04,3.70,abaxial,stomates
4,Ceratostigma_minus,3-07-400x-2-01,3.70,adaxial,stomates
...,...,...,...,...,...
76,Hemerocallis_fulva,7-24-400x-1-05,7.24,abaxial,longues
77,Jasminum fruticans,8-06-700x-2-01,8.60,NaN,puzzle
78,Jasminum fruticans,8-06-700x-2-02,8.60,NaN,puzzle
79,Jasminum fruticans,8-06-700x-2-03,8.60,NaN,puzzle


In [261]:
#here we import the csv file that we created in excel as a dataframe. It's the csv file of the first point of the
    #To-do list of "How cells are connected ?".

csv_to_df('FDV_database/cells_connections.csv',df_cells_connections)

,image_name,nb_non_triangular_junctions,total_nb_junctions,csv_file
0,3-07-400x-1-01,NaN,NaN,non_triangular_junctions-3-07-400x-1-01.csv
1,3-07-400x-1-02,NaN,NaN,non_triangular_junctions-3-07-400x-1-02.csv
2,3-07-400x-1-03,NaN,NaN,non_triangular_junctions-3-07-400x-1-03.csv
3,3-07-400x-1-04,NaN,NaN,non_triangular_junctions-3-07-400x-1-04.csv
4,3-07-400x-2-01,NaN,NaN,non_triangular_junctions-3-07-400x-2-01.csv
...,...,...,...,...
76,7-24-400x-1-05,NaN,NaN,non_triangular_junctions-7-24-400x-1-05.csv
77,8-06-700x-2-01,NaN,NaN,non_triangular_junctions-8-06-700x-2-01.csv
78,8-06-700x-2-02,NaN,NaN,non_triangular_junctions-8-06-700x-2-02.csv
79,8-06-700x-2-03,NaN,NaN,non_triangular_junctions-8-06-700x-2-03.csv


In [256]:
#As the column "image_name" are empty, we take the values of the column "cell_fname" of the global_df_cleaned2 
    #dataframe and use the panda function .assign to put them in the column "image_name" of the cells_connections
    #dataframe.
def assign_column(df1,df2,column_name):

    df1 = df1.assign(image_name = df2[column_name])
    return df1

In [257]:
assign_column(df_cells_connections,global_df_cleaned2,'cell_fname')

,image_name,nb_non_triangular_junctions,total_nb_junctions,csv_file
0,3-07-400x-1-01,NaN,NaN,non_triangular_junctions-3-07-400x-1-01.csv
1,3-07-400x-1-02,NaN,NaN,non_triangular_junctions-3-07-400x-1-02.csv
2,3-07-400x-1-03,NaN,NaN,non_triangular_junctions-3-07-400x-1-03.csv
3,3-07-400x-1-04,NaN,NaN,non_triangular_junctions-3-07-400x-1-04.csv
4,3-07-400x-2-01,NaN,NaN,non_triangular_junctions-3-07-400x-2-01.csv
...,...,...,...,...
76,7-24-400x-1-05,NaN,NaN,non_triangular_junctions-7-24-400x-1-05.csv
77,8-06-700x-2-01,NaN,NaN,non_triangular_junctions-8-06-700x-2-01.csv
78,8-06-700x-2-02,NaN,NaN,non_triangular_junctions-8-06-700x-2-02.csv
79,8-06-700x-2-03,NaN,NaN,non_triangular_junctions-8-06-700x-2-03.csv


In [259]:
#in this dataframe, we have less images, so we deleted the lines with NaN values in the column "image_name"

clean_df(df_cells_connections,df_cells_connections,'image_name')

,image_name,nb_non_triangular_junctions,total_nb_junctions,csv_file
0,3-07-400x-1-01,NaN,NaN,non_triangular_junctions-3-07-400x-1-01.csv
1,3-07-400x-1-02,NaN,NaN,non_triangular_junctions-3-07-400x-1-02.csv
2,3-07-400x-1-03,NaN,NaN,non_triangular_junctions-3-07-400x-1-03.csv
3,3-07-400x-1-04,NaN,NaN,non_triangular_junctions-3-07-400x-1-04.csv
4,3-07-400x-2-01,NaN,NaN,non_triangular_junctions-3-07-400x-2-01.csv
...,...,...,...,...
76,7-24-400x-1-05,NaN,NaN,non_triangular_junctions-7-24-400x-1-05.csv
77,8-06-700x-2-01,NaN,NaN,non_triangular_junctions-8-06-700x-2-01.csv
78,8-06-700x-2-02,NaN,NaN,non_triangular_junctions-8-06-700x-2-02.csv
79,8-06-700x-2-03,NaN,NaN,non_triangular_junctions-8-06-700x-2-03.csv


In [260]:
#here I tried to replace "image_name" in 'non_triangular_junctions-image_name.csv' with a for loop but It was
    #easier to use excel to do it. That is why I export the dataframe in csv in the next code cell.

'''
index = 0
for index in range(0,len(df_cells_connections),1):
    df_cells_connections._get_value(index, 'csv_file') == "non_triangular_junctions-"+df_cells_connections._get_value(0, 'image_name')+".csv"
df_cells_connections
'''

'\nindex = 0\nfor index in range(0,len(df_cells_connections),1):\n    df_cells_connections._get_value(index, \'csv_file\') == "non_triangular_junctions-"+df_cells_connections._get_value(0, \'image_name\')+".csv"\ndf_cells_connections\n'

In [262]:
def df_to_csv(df,csv):
    #the csv argument need to be a file name.csv between ""

    df.to_csv(csv, index=False)
    return csv

In [264]:
df_to_csv(df_cells_connections,'cells_connections.csv')

'cells_connectionss.csv'

## Segmentation (sans watershed ou label pour question 3) et sauvegarde des labels

### I use the code did by Inès to get all the final masks for each categories without the colors (without watershed or labels)

In [82]:
def segmentation_puzzle_without_watershed(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    return mask

In [83]:
def segmentation_puzzle_without_watershed(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    return mask

In [84]:
def segmentation_longue_without_label(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    return mask

In [85]:
def segmentation_stomates_without_label(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    footprint = morphology.disk(4)
    mask_eroted = morphology.binary_erosion(mask, footprint)
    
    return mask_eroted

In [86]:
def segmentation_veine_without_label(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    footprint = morphology.disk(4)    

    mask_dilation = morphology.binary_dilation(mask , footprint)
    mask_eroted = morphology.binary_erosion(mask_dilation, footprint)
    mask_fill = ndi.binary_fill_holes(mask_eroted)

    #viewer.add_image(mask_fill)
    
    return mask_fill

In [87]:
def segmentation_croco_without_label(path_img) :

    img = imread(path_img)
    img_gray = skimage.color.rgb2gray(img)

    #viewer = napari.Viewer()

    #viewer.add_image(img_gray.astype(np.uint8), name = 'astype')
    #viewer.add_image(img_gray*255, name = '255')


    footprint = disk(15.0)
    img_median_blur = skimage.filters.median(img_gray, footprint)


    local_threshold = filters.threshold_local(img_median_blur, block_size=35)
    mask = img_median_blur > local_threshold

    mask = morphology.remove_small_holes(mask.astype(bool), 2500)
    mask = morphology.remove_small_objects(mask.astype(bool), 500)


    #viewer.add_image(mask)
    
    footprint = morphology.disk(8)    

    mask_dilation = morphology.binary_dilation(mask , footprint)
    mask_eroted = morphology.binary_erosion(mask_dilation, footprint)
    mask_fill = ndi.binary_fill_holes(mask_eroted)

    #viewer.add_image(mask_fill)
    
    return mask_fill

In [16]:
# images longues file
for image in glob.glob('/home/criuser/Téléchargements/FDV_database/FDV_cell_database_cleaned/longues/*.tif') : 
    label = segmentation_longue_without_label(image)
    splitted_path = image.split("/")
    name = splitted_path[-1]
    tifffile.imwrite('sauvegarde/longues/' + name,label)

In [17]:
# images croco file
for image in glob.glob('/home/criuser/Téléchargements/FDV_database/FDV_cell_database_cleaned/croco/*.tif') : 
    label = segmentation_croco_without_label(image)
    splitted_path = image.split("/")
    name = splitted_path[-1]
    tifffile.imwrite('sauvegarde/croco/' + name,label)

In [18]:
# images puzzle file
for image in glob.glob('/home/criuser/Téléchargements/FDV_database/FDV_cell_database_cleaned/puzzle/*.tif') : 
    label = segmentation_puzzle_without_watershed(image)
    splitted_path = image.split("/")
    name = splitted_path[-1]
    tifffile.imwrite('sauvegarde/puzzle/' + name,label)

In [19]:
# images veines file
for image in glob.glob('/home/criuser/Téléchargements/FDV_database/FDV_cell_database_cleaned/veines/*.tif') : 
    label = segmentation_veine_without_label(image)
    splitted_path = image.split("/")
    name = splitted_path[-1]
    tifffile.imwrite('sauvegarde/veines/' + name,label)

In [20]:
# images stomates file
for image in glob.glob('/home/criuser/Téléchargements/FDV_database/FDV_cell_database_cleaned/stomates/*.tif') : 
    label = segmentation_stomates_without_label(image)
    splitted_path = image.split("/")
    name = splitted_path[-1]
    tifffile.imwrite('sauvegarde/stomates/' + name,label)

Test :

In [21]:
viewer = napari.Viewer()

#### Create a function to reverse the mask and get an image with the membranes only

In [88]:
#reverse the mask
def inverse_mask(image):
    
    image = ~image #here we reverse the mask by change 0 to 1 and change 1 to 0
    return image

#### Refine the membranes in one pixel size line to count the different surrounding pixels of each pixel

In [120]:
#perform skeletonization to refine the membranes and get one pixel size lines (it will permit to compare the pixels
#and know if there are junctions)

skeleton = skeletonize(image)

#### Turn the array in an array with just 0 and 1 

In [310]:
#turn the array in an array with just 0 (if pixel < 3) and 1 (if pixel > 3)
def turn_array_0_1(skeleton):
    
    for pixel in skeleton:
        if pixel.any() < 3:
            pixel == 0
        else:
            pixel == 1
    new_array_TF = skeleton
    
    new_array = new_array_TF.astype(int)
    
    return new_array
    
    #labels = skimage.morphology.label(new_array)

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/croco/*.tif') : 
    #df_non_triangular_junctions_image = pd.read_csv(image)
    img = imread(image)
    viewer.add_image(img)

#images longues file

'''
import sys

class recursion_depth:
    def __init__(self, limit):
        self.limit = limit
        self.default_limit = sys.getrecursionlimit()

    def __enter__(self):
        sys.setrecursionlimit(self.limit)

    def __exit__(self, type, value, traceback):
        sys.setrecursionlimit(self.default_limit)
    
    __init__(self, 2000)
    __enter__(2000)
    __exit__(self, type, value, traceback)
    
'''

    for image in viewer.layers: 

        #reverse the mask
        mask_reversed = inverse_mask(image.data)
        viewer.add_image(mask_reversed)

        #perform skeletonization
        skeleton = skeletonize(viewer.layers['mask_reversed'])    
        viewer.add_image(skeleton)

        #turn the array in an array with just 0 (if pixel < 3) and 1 (if pixel > 3)
        turned_array = turn_array_0_1(skeleton.data)


        splitted_path = image.split("/")
        name = splitted_path[-1]
        df_non_triangular_junctions_image = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')

In [352]:
#image longues file

mask = []

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/longues/*.tif') : 
    img = imread(image)
    
    #reverse the mask
    mask_reversed = inverse_mask(img)
    mask.append(mask_reversed)
    
    skeletonized = []

for array in mask:
    #perform skeletonization
    skeleton = skeletonize(array)
    skeletonized.append(skeleton)
    
    turned_array_list = []

for arrays in skeletonized:
    turned_array = turn_array_0_1(arrays)
    turned_array_list.append(turned_array)
    
    csv_non_tri_junctions_longues = []

for image in df_cells_connections['image_name']:

    splitted_path = image.split("/")
    name = splitted_path[-1]
    name = name.replace('tif','csv')
    df_non_triangular_junctions_image_longues = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')
    csv_non_tri_junctions_longues.append(df_non_triangular_junctions_image_longues)
df_non_triangular_junctions_image_longues

,Unnamed: 0,label,centroid-0,centroid-1
0,0,1,25.664792,15.395951
1,1,2,39.529279,179.509141
2,2,3,19.789255,320.584615
3,3,4,27.378383,577.721277
4,4,5,19.470324,875.340827
...,...,...,...,...
112,112,113,1163.332464,108.205283
113,113,114,1184.034413,16.052632
114,114,115,1180.649596,202.217559
115,115,116,1177.482280,651.997579


In [353]:
#images puzzle file

mask = []

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/puzzle/*.tif') : 
    img = imread(image)
    
    #reverse the mask
    mask_reversed = inverse_mask(img)
    mask.append(mask_reversed)
    
    skeletonized = []

for array in mask:
    #perform skeletonization
    skeleton = skeletonize(array)
    skeletonized.append(skeleton)
    
    turned_array_list = []

for arrays in skeletonized:
    turned_array = turn_array_0_1(arrays)
    turned_array_list.append(turned_array)
    
    csv_non_tri_junctions_puzzle = []

for image in df_cells_connections['image_name']:

    splitted_path = image.split("/")
    name = splitted_path[-1]
    name = name.replace('tif','csv')
    df_non_triangular_junctions_image_puzzle = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')
    csv_non_tri_junctions_puzzle.append(df_non_triangular_junctions_image_puzzle)
#df_non_triangular_junctions_image_puzzle

In [354]:
#images veines file

mask = []

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/veines/*.tif') : 
    img = imread(image)
    
    #reverse the mask
    mask_reversed = inverse_mask(img)
    mask.append(mask_reversed)
    
    skeletonized = []

for array in mask:
    #perform skeletonization
    skeleton = skeletonize(array)
    skeletonized.append(skeleton)
    
    turned_array_list = []

for arrays in skeletonized:
    turned_array = turn_array_0_1(arrays)
    turned_array_list.append(turned_array)
    
    csv_non_tri_junctions_veines = []

for image in df_cells_connections['image_name']:

    splitted_path = image.split("/")
    name = splitted_path[-1]
    name = name.replace('tif','csv')
    df_non_triangular_junctions_image_veines = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')
    csv_non_tri_junctions_veines.append(df_non_triangular_junctions_image_veines)
#df_non_triangular_junctions_image_veines

In [355]:
#images stomates file

mask = []

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/stomates/*.tif') : 
    img = imread(image)
    
    #reverse the mask
    mask_reversed = inverse_mask(img)
    mask.append(mask_reversed)
    
    skeletonized = []

for array in mask:
    #perform skeletonization
    skeleton = skeletonize(array)
    skeletonized.append(skeleton)
    
    turned_array_list = []

for arrays in skeletonized:
    turned_array = turn_array_0_1(arrays)
    turned_array_list.append(turned_array)
    
    csv_non_tri_junctions_stomates = []

for image in df_cells_connections['image_name']:

    splitted_path = image.split("/")
    name = splitted_path[-1]
    name = name.replace('tif','csv')
    df_non_triangular_junctions_image_stomates = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')
    csv_non_tri_junctions_stomates.append(df_non_triangular_junctions_image_stomates)
#df_non_triangular_junctions_image_stomates

In [356]:
#images croco file

mask = []

for image in glob.glob('/home/criuser/Téléchargements/sauvegarde/croco/*.tif') : 
    img = imread(image)
    
    #reverse the mask
    mask_reversed = inverse_mask(img)
    mask.append(mask_reversed)
    
    skeletonized = []

for array in mask:
    #perform skeletonization
    skeleton = skeletonize(array)
    skeletonized.append(skeleton)
    
    turned_array_list = []

for arrays in skeletonized:
    turned_array = turn_array_0_1(arrays)
    turned_array_list.append(turned_array)
    
    csv_non_tri_junctions_croco = []

for image in df_cells_connections['image_name']:

    splitted_path = image.split("/")
    name = splitted_path[-1]
    name = name.replace('tif','csv')
    df_non_triangular_junctions_image_croco = pd.read_csv(
        '/home/criuser/Téléchargements/FDV_database/Centroid/non_triangular_junctions-'+name+'.csv')
    csv_non_tri_junctions_croco.append(df_non_triangular_junctions_image_croco)
#df_non_triangular_junctions_image_croco

In [364]:
#label
#region prop
#fonction obscure
#centroid


#### Create centroid  and count them

In [357]:
#Inès already wrote a code to create all the centroid, count them, and save these data in a csv file

props = measure.regionprops_table(
    labels,
    properties=['label', 'centroid'] 
    )

df_non_triangular_junctions_image = pd.DataFrame(props)
total_nb_junctions = len(df_non_triangular_junctions['label'])
total_nb_junctions

NameError: name 'labels' is not defined

#### Get euclidian distance using NearestNeighbors function

In [398]:
#get euclidian distance, finally we use the functions NearestNeighbors

#euclidian_dist = new_layer.data.linalg.norm(a-b)

In [358]:
def non_triangular_junctions(df_non_triangular_junctions):
    
    #creating a list with all the centroids coordinates:
    centroids = np.array([list(df_non_triangular_junctions['centroid-0']),list(
        df_non_triangular_junctions['centroid-1'])]).T
    
    #find nearest neighbors :

    number = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(centroids)
    distances, indices = number.kneighbors(centroids)

    #determine non triangular junctions :
    non_tri_junctions = np.unique(indices[distances[:, 1] < 5])
    
    #get the centroid coordinates of non triangular junctions :
    df_non_tri_junctions = pd.DataFrame()
    df_non_tri_junctions = df_non_triangular_junctions[df_non_triangular_junctions['label'].isin(non_tri_junctions)]
    
    return (df_non_tri_junctions)

In [359]:
for arrays in csv_non_tri_junctions_longues:
    non_triangular_junctions(arrays)
non_triangular_junctions(arrays)

,Unnamed: 0,label,centroid-0,centroid-1


In [360]:
for arrays in csv_non_tri_junctions_puzzle:
    non_triangular_junctions(arrays)
non_triangular_junctions(arrays)

,Unnamed: 0,label,centroid-0,centroid-1


In [361]:
for arrays in csv_non_tri_junctions_stomates:
    non_triangular_junctions(arrays)
non_triangular_junctions(arrays)

,Unnamed: 0,label,centroid-0,centroid-1


In [362]:
for arrays in csv_non_tri_junctions_veines:
    non_triangular_junctions(arrays)
non_triangular_junctions(arrays)

,Unnamed: 0,label,centroid-0,centroid-1


In [367]:
for arrays in csv_non_tri_junctions_croco:
    non_triangular_junctions(arrays)
non_triangular_junctions(arrays)

,Unnamed: 0,label,centroid-0,centroid-1


### Create a bar plot showing in x the species and in y the measured (non_triangular / total) value for each image

In [364]:
 
# creating the bar plot
plt.bar(x=global_df_cleaned2['plant_name'],
        y=df_cells_connections['nb_non_triangular_junctions']/df_cells_connections['total_nb_junctions']
        ,color ='maroon',width = 0.4)

plt.figure(figsize=(10,5))
plt.xlabel("Plant_name")
plt.ylabel("Number of non_triangular_junctions")
plt.title("Species' names in function of the measured (non_triangular / total) value for each image")
plt.show()

TypeError: bar() missing 1 required positional argument: 'height'

In [ ]:
#how to use de ma question + clean csv clean data + check the PEP8 writing conditions